In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from typing import Counter
from NBClassifier import NBClassifier

In [63]:
df = pd.read_csv('mmm.csv')
df.columns = ['Label', 'Mail']
print(df.shape)
df.head()

(5171, 2)


,Label,Mail
0,0,Subject: enron methanol ; meter # : 988291\r\n...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\r\nthis deal is t...


In [64]:
df.drop_duplicates(inplace = True)
df.shape

(4993, 2)

In [65]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [66]:
df['Mail'] = df['Mail'].apply(process_text)
df['Mail']

0       [Subject, enron, methanol, meter, 988291, foll...
1       [Subject, hpl, nom, january, 9, 2001, see, att...
2       [Subject, neon, retreat, ho, ho, ho, around, w...
3       [Subject, photoshop, windows, office, cheap, m...
4       [Subject, indian, springs, deal, book, teco, p...
                              ...                        
5165    [Subject, fw, crosstex, energy, driscoll, ranc...
5166    [Subject, put, 10, ft, transport, volumes, dec...
5167    [Subject, 3, 4, 2000, following, noms, hpl, ta...
5169    [Subject, industrial, worksheets, august, 2000...
5170    [Subject, important, online, banking, alert, d...
Name: Mail, Length: 4993, dtype: object

In [67]:
randomized = df.sample(frac=1)
train_size = round(len(randomized) * 0.8)
train = randomized[:train_size].reset_index(drop=True)
test = randomized[train_size:].reset_index(drop=True)
print(train.shape)
print(test.shape)

(3994, 2)
(999, 2)


In [68]:
bag_word = []
testmails= test['Mail']
test_labels = test['Label']
testmails2 = []
trainmails = train['Mail']
train_labels = train['Label']
trainmails2= []

In [69]:
for doc in trainmails:
    words = doc
    bag_word += words

bag_word = set(bag_word)
bag_word = list(bag_word)

def tovector(mail):   
    splitmail = mail
    mvec = np.zeros(len(bag_word))
    count = Counter(splitmail)
    for i, word in enumerate(bag_word):
                mvec[i]+=count[word]
    
    return mvec

In [70]:
for i,mail in enumerate(testmails):

    testmails2.append(tovector(mail))

for i,mail in enumerate(trainmails):

    trainmails2.append(tovector(mail))

In [71]:
d = {'Mail': trainmails2, 'Label': train_labels}
Trainingdata = pd.DataFrame(data=d)
d = {'Mail': testmails2, 'Label': test_labels}
Testdata = pd.DataFrame(data=d)

print(Testdata)

                                                  Mail  Label
0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
..                                                 ...    ...
994  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
995  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0
996  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
997  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      1
998  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...      0

[999 rows x 2 columns]


In [79]:
model= NBClassifier()
model.fit(Trainingdata['Mail'],Trainingdata['Label'])
result = model.predict(Testdata)

model.score(Testdata)

0.978978978978979

In [6]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', MultinomialNB() )])

pipe.fit(train['Mail'], train['Label'])

Pipeline(steps=[('vectorize', CountVectorizer(token_pattern='\\b\\w+\\b')),
                ('classifier', MultinomialNB())])

In [7]:
y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.978978978978979


In [8]:
test['predict'] = y_predict
test

,Label,Mail,predict
0,0,Subject: - - - - - - - - - - - - - - - - - - -...,0
1,1,"Subject: colloquy\r\nhi , this is christine re...",1
2,1,"Subject: vicodin , via - gra are che . ap here...",1
3,0,"Subject: hpl nom for august 11 , 2000\r\noops ...",0
4,0,Subject: re : boat\r\ni believe the boat is 18...,0
...,...,...,...
994,0,"Subject: hpl nom for august 12 , 2000\r\n( see...",0
995,1,"Subject: approval # 5146\r\nhello ,\r\nwe sent...",1
996,1,"Subject: down . load - dvd , mp 3 , music , pl...",1
997,0,Subject: july transport usage tickets\r\ni hav...,0


In [9]:
accuracy = sum(test.Label == test.predict)
accuracy

978

In [10]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', LogisticRegression())])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))


0.978978978978979


c:\Users\LUONG.VB\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', SVC(kernel = 'linear'))])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.968968968968969


In [12]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', RandomForestClassifier(n_estimators=100))])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.96996996996997


In [15]:
pipe = Pipeline(steps=[('vectorize', CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b')),
                       ('classifier', MLPClassifier())])

pipe.fit(train['Mail'], train['Label'])

y_predict = pipe.predict(test['Mail'])

print(accuracy_score(test['Label'], y_predict))

0.980980980980981
